In [1]:
import openai
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("openai_api_key")

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter


model = ChatOpenAI(model="gpt-4o-mini", temperature = 1)

In [9]:
prompt_1 = ChatPromptTemplate.from_template("name one country with the color {color} in its flag")
prompt_2 = ChatPromptTemplate.from_template("name and describe 3 well-known {noun} from {country}.")

chain_1 = prompt_1 | model | StrOutputParser()

#chain_1.invoke({"color":"orange"})

In [8]:
chain_1.invoke({"color":"green"})

'One country with green in its flag is Brazil.'

In [10]:
prompt_2.messages[0].prompt.input_variables

['country', 'noun']

## A chain.  The output to chain_1 becomes the input to chain_2

In [13]:
chain_2 = ({"country":chain_1, "noun":itemgetter("noun")} | prompt_2 | model | StrOutputParser())

answer = chain_2.invoke({"color":"orange", "noun":"sports"})
print(answer)

The flag of the Netherlands features a horizontal tricolor of red, white, and blue, but orange is also a significant national color associated with Dutch identity, largely due to the royal House of Orange-Nassau. Here are three well-known sports popular in the Netherlands:

1. **Field Hockey**: Field hockey is one of the most successful team sports in the Netherlands. The Dutch national teams, both men’s and women’s, have a storied history, consistently performing well in international competitions, including the Olympics and World Cups. The sport is played on grass or turf, where teams try to score goals by hitting a ball into the opponent's net using curved sticks. The Dutch women’s team, in particular, is renowned for its dominance in the sport, with numerous European titles and World Cup victories.

2. **Speed Skating**: The Netherlands is famous for its achievements in speed skating, especially in long track events. The sport holds a special place in Dutch culture, particularly du

In [14]:
import pandas as pd

df = pd.read_csv('home_goods_reviews.csv')
df[0:5]

,product,product review
0,Cotton Bath Towels,These towels are incredibly soft and absorbent...
1,Stainless Steel Cookware Set,High-quality cookware that heats evenly. Makes...
2,Memory Foam Pillow,This pillow has improved my sleep quality sign...
3,LED Desk Lamp,Bright and adjustable lighting. Ideal for work...
4,Automatic Coffee Maker,Makes a great cup of coffee with minimal effor...


In [15]:
prompt_companyName = ChatPromptTemplate.from_template(
    "What is the one best name to describe a company that makes {products}?"
)

In [16]:
chain_a = prompt_companyName | model | StrOutputParser()


company_name_suggestion = chain_a.invoke({'products': df['product']})
print(company_name_suggestion)

A suitable name for a company that offers a diverse range of household and lifestyle products, as listed, could be **"Home Harmony"**. This name conveys a sense of comfort, style, and utility, reflecting the variety of products aimed at enhancing the home environment.


In [17]:
prompt_companyDescription = ChatPromptTemplate.from_template(
    "Write a description in marketing language using {number} of words for the following company:{company_name}"
)

In [18]:
chain_b = {"company_name" : chain_a, "number": itemgetter("number")} | prompt_companyDescription | model | StrOutputParser()

chain_b.invoke({'products': df['product'], "number": 30}) 

'Welcome to HomeHarmony—your ultimate destination for diverse, high-quality home and kitchen products. Transform your living space into a sanctuary of comfort and style. Experience harmony, functionality, and elegance today!'

## A "branching" system.  Two prompts are run in parallel and then the results merged before returning output to the user.

In [5]:
prompt_1 = ChatPromptTemplate.from_template("answer the question {question} from a conservative, risk-averse standpoint")
prompt_2 = ChatPromptTemplate.from_template("answer the question {question} from a growth minded, risk-loving standpoint.")

prompt_synthesize = ChatPromptTemplate.from_template(
"""Given the question: {question} and the below two answers:
Answer 1:
{answer1}

Answer 2:
{answer2}

Combine the viewpoints of two answers and form a coherent combined answer.
"""
)

model1 = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
model2 = ChatOpenAI(model="gpt-4o-mini", temperature=1)
model3 = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain1 = prompt_1 | model1 | StrOutputParser()
chain2 = prompt_2 | model2 | StrOutputParser()
chain3 = prompt_synthesize | model3 | StrOutputParser()


In [6]:
question = "how should I invest for retirement?"

Inputs to final chain ("combined_answer") are the outputs from chain 1 ("answer1) and chain 2 ("answer2) and the original question ("quetstion").  RunnablePassthrough() allows an input to be passed unchanged to downstream chains.

In [12]:
combined_chain = {
    "question": RunnablePassthrough(),
    "answer1": chain1,
    "answer2": chain2,
} | chain3

combined_answer = combined_chain.invoke(question)

In [13]:
print(combined_answer)

Investing for retirement can be approached from both conservative and growth-oriented perspectives, and a balanced strategy that incorporates elements from both can help you achieve your retirement goals while managing risk. Here’s a comprehensive approach to consider:

### 1. **Diversified Investment Portfolio**:
   - **Equities**: Allocate a portion of your portfolio to growth stocks, including technology and small-cap stocks, which have the potential for substantial returns. Consider international stocks and emerging markets for additional growth opportunities, but be mindful of the associated risks.
   - **Fixed Income Investments**: Balance your equity investments with a diversified bond portfolio, including government bonds, municipal bonds, and investment-grade corporate bonds. This can provide stability and income, helping to preserve capital.

### 2. **Exchange-Traded Funds (ETFs) and Mutual Funds**:
   - Utilize ETFs and mutual funds that focus on both growth and conservative

## A "routing" system.  First prompt evaluates user input and conditionally chooses path for further processing.

In [20]:
email = """
    Had a very horrible experience. Remote wouldn’t connect properly even with strong WiFi connection.
    So I returned the item, paid for shipping. I purchased this item for $312, and was expecting entire amount refunded. 
    But just to find out they charged me $156 “Restocking fee” and additional shipping fee.
    There was no indication of such when purchasing the item. I was unaware of such fees and I
    would have never purchased if I knew I was going to lose half of the money.
    Been trying to resolve this issue with Amazon and there isn’t much they can do.
    """

In [26]:
email = """
The Playstation Portal is awesome!  I love it!
    """

In [21]:
prompt_mood =  PromptTemplate.from_template(
        """Given the user email below, classify it as either being about "unhappy", "happy" or "other".  Do not respond with more than one word.
            {email}
        """
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
    
chain_detect_mood = prompt_mood | model  | StrOutputParser()
chain_detect_mood.invoke({"email": email})

'unhappy'

Create alternative prompts, describing response elements depending on the mood of the incoming customer email.

In [22]:
chain_unhappy = PromptTemplate.from_template(
    """You are responding to an unhappy customer experience.  Respond in a polite and empathic tone.  Be sure to:
    Acknowledge the customer’s frustration.
    Apologize for the inconvenience.
    Offer a solution or ask for more information to resolve the issue.
    Mention the negative issues they raised specifically so they know they were heard.

Email: {email}
"""
) | model | StrOutputParser()


chain_happy = PromptTemplate.from_template(
    """You are responding to an happy customer experience.  Respond in a polite and empathic tone.  Be sure to:
    Express gratitude for the positive feedback.
    Encourage further engagement.
    Extend an invitation to share their experience with others.  
    Mention any particular positive issues they raised specifically so they know they were heard.

Email: {email}
"""
) | model | StrOutputParser()


chain_other = PromptTemplate.from_template(
    """You are responding to an customer experience.  Respond in a polite and empathic tone.

Email: {email}
"""
) | model | StrOutputParser()

In [23]:
# custom route function

def route(info):
    if "unhappy" in info["mood"].lower():
        return chain_unhappy
    elif "happy" in info["mood"].lower():
        return chain_happy
    else:
        return chain_other

In [24]:
# Inputs to chain -"mood" from chain_detect_mood, and "email" - sent to route() function.

from langchain_core.runnables import RunnableLambda

full_chain = {"mood": chain_detect_mood, "email": lambda x: x["email"]} | RunnableLambda(
    route
)

In [27]:
print(full_chain.invoke({"email": email}))

Subject: Thank You for Your Wonderful Feedback!

Dear [Customer's Name],

Thank you so much for taking the time to share your thoughts about the PlayStation Portal! We're thrilled to hear that you love it and that it has enhanced your gaming experience.

Your enthusiasm truly brightens our day, and we appreciate your positive feedback about its features. It's fantastic to know that the Portal has met your expectations and brought you joy.

We would love to hear more about your experiences with the Portal! Please feel free to reach out anytime with your thoughts or suggestions. Additionally, if you know anyone who might be interested in the PlayStation Portal, we encourage you to share your experience with them. Your insights could help others discover the excitement you’ve found!

Thank you once again for your kind words. We’re here to support you on your gaming journey!

Warm regards,

[Your Name]  
[Your Position]  
[Your Company]  
[Contact Information]  
